In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

In [3]:
from interface.TimitInterface import TimitInterface
from phoneme import initialize_model, get_needed_data

c:\Users\kavya\anaconda3\envs\accent-capstone\lib\site-packages\transformers\modeling_utils.py:1331: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(re

In [4]:
#processor, model, device = initialize_model()
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureExtractor(
      (conv_layers): ModuleList(
        (0): Wav2Vec2GroupNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
        )
        (1-4): 4 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        )
        (5-6): 2 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
        )
      )
    )
    (feature_projection): Wav2Vec2FeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projection): Linear(in_features=512, out_features=768, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): Wav2Vec2Encoder(
      (pos_conv_embed): Wav2Vec2PositionalConvEmbedding(
        (conv): Conv1d(768, 768, kernel_size=

In [5]:
dataset = TimitInterface()

0it [00:00, ?it/s]

Map:   0%|          | 0/2688 [00:00<?, ? examples/s]

Map:   0%|          | 0/336 [00:00<?, ? examples/s]

Map:   0%|          | 0/336 [00:00<?, ? examples/s]

In [6]:
val_data = dataset.valid
train_data = dataset.train

In [7]:
data = {}
#load data

In [8]:
train_output = list(map(lambda v: (v['audio_file'], v['word_file'], v['phonetic_file']), train_data.values()))
val_output = list(map(lambda v: (v['audio_file'], v['word_file'], v['phonetic_file']), val_data.values()))

In [10]:
print(len(train_output))

2688


In [11]:
print(len(val_output))

336


In [9]:
# Second define your dataloader to give inputs, targets, input_length and target_length
train_dataloader = []
val_dataloader = []
for wav_path, txt_path, phn_path in train_output:
    train_dataloader.append(get_needed_data(wav_path, txt_path, phn_path))

for wav_path, txt_path, phn_path in val_output:
    val_dataloader.append(get_needed_data(wav_path, txt_path, phn_path))

ctc_loss = nn.CTCLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

num_epochs = 1 #3
max_grad_norm = 1.0

c:\Users\kavya\anaconda3\envs\accent-capstone\lib\site-packages\transformers\feature_extraction_utils.py:158: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:281.)
  tensor = as_tensor(value)


In [30]:
count = 0
for epoch in range(num_epochs):
    model.train()
    for _, _, inputs, targets, _, target_lengths in train_dataloader:
        count += 1
        print(count)
        optimizer.zero_grad()
        #print(inputs)
        outputs = model(inputs).logits
        outputs = outputs.transpose(0, 1)
        #print(outputs.shape)
        #print(outputs)
        #print(outputs[0])

        if torch.isnan(outputs).any():
            print("NaN detected in outputs")
            continue
        
        input_lengths = torch.tensor([outputs.shape[0]], dtype=torch.int32)
        target_lengths = target_lengths.to(torch.int32)

        # Calculate log probabilities
        log_probs = torch.log_softmax(outputs, dim=-1)
        #print(f'log_probs: {log_probs}')
        # Get predicted indices
        predicted_indices = torch.argmax(log_probs, dim=-1)  # (seq_len, batch_size)
        predicted_indices = predicted_indices.reshape(1, input_lengths)
        # Print or store the phoneme predictions
        #print("Phoneme Predictions:", predicted_indices)
        
        #print(targets)
        loss = ctc_loss(outputs, targets, input_lengths, target_lengths)
        #print(f'loss: {loss}')
        loss.backward()
        if count % 10 == 0:
            print(predicted_indices)
            print(loss)
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        
        optimizer.step()
        if count == 200:
            break
    
    # Validation
    model.eval()
    val_loss = 0
    count = 0
    with torch.no_grad():
        for _, _, inputs, targets, _, target_lengths in val_dataloader:
            count += 1
            print(count)
            outputs = model(inputs).logits
            outputs = outputs.transpose(0, 1)
            input_lengths = torch.tensor([outputs.shape[0]], dtype=torch.int32)
            target_lengths = target_lengths.to(torch.int32)
            loss = ctc_loss(outputs, targets, input_lengths, target_lengths)
            val_loss += loss.item()
            if count == 50:
                break
    
    val_loss /= len(val_dataloader)
    print(f"Epoch {epoch + 1}/{num_epochs}, Validation Loss: {val_loss:.4f}")

1
2
3
4
5
6
7
8
9
10
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
tensor(2.9985, grad_fn=<MeanBackward0>)
11


KeyboardInterrupt: 